# Training of Siamese Networks

In [1]:
import numpy as np
import os
import pandas as pd

from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Sequential

from Siamese_model import SiameseTrainer

from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [2]:
def create_pairs(x, digit_indices, num_classes):
    """Creates a balanced dataset of pairs for Siamese networks."""
    pairs = []
    labels = []

    # n: smallest number of instances among all classes in the dataset
    n = min([len(digit_indices[d]) for d in range(num_classes)]) - 1
    
    for d in range(num_classes):
        for i in range(n):
            z1, z2 = digit_indices[d][i], digit_indices[d][i + 1]
            pairs += [[x[z1], x[z2]]]
            inc = np.random.randint(1, num_classes)
            dn = (d + inc) % num_classes
            z1, z2 = digit_indices[d][i], digit_indices[dn][i]
            pairs += [[x[z1], x[z2]]]
            labels += [1, 0]
    return np.array(pairs), np.array(labels)

In [3]:
def train(self, x_train, y_train, x_val, y_val, batch_size=32, epochs=10):
    # Create digit_indices for training and validation sets
    digit_indices_train = [np.where(y_train == i)[0] for i in range(self.num_classes)]
    digit_indices_val = [np.where(y_val == i)[0] for i in range(self.num_classes)]

    # Create training pairs
    tr_pairs, tr_y = create_pairs(x_train, digit_indices_train, self.num_classes)

    # Create validation pairs
    val_pairs, val_y = create_pairs(x_val, digit_indices_val, self.num_classes)

    # Train the model
    self.history = self.model.fit([tr_pairs[:, 0], tr_pairs[:, 1]], tr_y,
                                  batch_size=batch_size,
                                  epochs=epochs,
                                  validation_data=([val_pairs[:, 0], val_pairs[:, 1]], val_y))


In [4]:
def load_and_preprocess_image(path, target_size=(96, 96)):
    img = load_img(path, target_size=target_size, color_mode='grayscale')
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img /= 255.0  # Normalize to [0,1]
    return img

# def load_pairs_and_labels(csv_path, image_base_path):
#     df = pd.read_csv(csv_path)
#     pairs = []
#     labels = df['label'].values
    
#     for idx, row in df.iterrows():
#         img1_path = os.path.join(image_base_path, row['image_1'])
#         img2_path = os.path.join(image_base_path, row['image_2'])
        
#         img1 = load_and_preprocess_image(img1_path)
#         img2 = load_and_preprocess_image(img2_path)
        
#         pairs.append([img1, img2])
    
#     pairs = np.array(pairs)
#     # Reshape to fit the Siamese network input
#     pairs = [pairs[:, 0].squeeze(), pairs[:, 1].squeeze()]
    
#     return pairs, labels

def load_pairs_and_labels(csv_path, image_base_path):
    df = pd.read_csv(csv_path)
    pair_images = []
    labels = df['label'].values
    
    for idx, row in df.iterrows():
        img1_path = os.path.join(image_base_path, row['image_1'])
        img2_path = os.path.join(image_base_path, row['image_2'])
        
        img1 = load_and_preprocess_image(img1_path)
        img2 = load_and_preprocess_image(img2_path)
        
        pair_images.append(np.concatenate([img1, img2], axis=0))
    
    # Convert list of pairs to a numpy array
    pairs = np.array(pair_images)
    
    # Since each image pair is concatenated along axis=0, we reshape to ensure
    # the final structure is compatible with what train_model expects
    pairs = pairs.reshape(-1, 2, *pairs.shape[2:])
    
    return pairs, labels


In [5]:
# Create a CNNModel object
def simple_cnn(input_shape, num_classes):
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2,2)),
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D((2,2)),
        Conv2D(128, (3,3), activation='relu'),
        MaxPooling2D((2,2)),
        Flatten(),
        Dense(1024, activation='relu'),
        Dense(128, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    return model

In [6]:
input_shape = (96, 96, 1)
num_classes = 3
epochs = 10

train_path = os.getcwd() + "/Split_Siamese/train.csv"
validation_path = os.getcwd() + "/Split_Siamese/validation.csv"
test_path = os.getcwd() + "/Split_Siamese/test.csv"

image_dir = '../OpenSARShip/Categories/'

In [7]:
model = simple_cnn(input_shape, num_classes)

In [8]:
# Define paths
image_base_path = '../OpenSARShip/Categories/'
train_csv_path = os.getcwd() + "/Split_Siamese/train_pairs.csv"
val_csv_path = os.getcwd() + "/Split_Siamese/val_pairs.csv"
test_csv_path = os.getcwd() + "/Split_Siamese/test_pairs.csv"

# Load and preprocess the data
print("Loading and preprocessing the data train")
train_pairs, train_labels = load_pairs_and_labels(train_csv_path, image_base_path)
print("Loading and preprocessing the data val")
val_pairs, val_labels = load_pairs_and_labels(val_csv_path, image_base_path)
print("Loading and preprocessing the data test")
test_pairs, test_labels = load_pairs_and_labels(test_csv_path, image_base_path)



Loading and preprocessing the data train
Loading and preprocessing the data val
Loading and preprocessing the data test


In [9]:
# print(type(train_pairs), type(train_labels))
# print(train_pairs.shape, train_labels.shape)

In [10]:
# Assuming `simple_cnn` and `SiameseTrainer` are defined as before
print("Creating the SiameseTrainer")
siamese_network = SiameseTrainer(base_model_func=simple_cnn, input_shape=(96, 96, 1), num_classes=num_classes)

# Compile the model
print("Compiling the model")
siamese_network.compile_model(optimizer='adam', loss='binary_crossentropy')

# Train the model
print("Training the model")
# siamese_network.train_model(train_pairs, train_labels, val_pairs, val_labels, epochs=10, batch_size=32)
siamese_network.train_model(train_pairs, train_labels, val_pairs, val_labels, epochs=10, batch_size=64)


# Plot training history
print("Plotting the training history")
siamese_network.plot_training()

# Evaluate the model
print("Evaluating the model")
siamese_network.evaluate_model(test_pairs, test_labels)


Creating the SiameseTrainer
Compiling the model
Training the model
Epoch 1/10
97/97 [==============================] - 1591s 16s/step - loss: 0.6890 - accuracy: 0.5928 - val_loss: 0.6906 - val_accuracy: 0.5392
Epoch 2/10
97/97 [==============================] - 1612s 17s/step - loss: 0.6820 - accuracy: 0.6010 - val_loss: 0.6892 - val_accuracy: 0.5392
Epoch 3/10
97/97 [==============================] - 1927s 20s/step - loss: 0.6763 - accuracy: 0.6036 - val_loss: 0.6857 - val_accuracy: 0.5524
Epoch 4/10
97/97 [==============================] - 1865s 19s/step - loss: 0.6722 - accuracy: 0.6036 - val_loss: 0.6866 - val_accuracy: 0.5470
Epoch 5/10
97/97 [==============================] - 1878s 19s/step - loss: 0.6693 - accuracy: 0.6033 - val_loss: 0.6870 - val_accuracy: 0.5456
Epoch 6/10
97/97 [==============================] - 1819s 19s/step - loss: 0.6669 - accuracy: 0.6037 - val_loss: 0.6851 - val_accuracy: 0.5523
Epoch 7/10
 6/97 [>.............................] - ETA: 32:54 - loss: 0.66

KeyboardInterrupt: 

In [ ]:
# # Assuming you have defined `simple_cnn` as before and have loaded your dataset
# siamese_network = SiameseTrainer(base_model_func=simple_cnn, input_shape=input_shape, num_classes=num_classes)

# # Compile the model with the desired optimizer, loss, and metrics
# siamese_network.compile_model(optimizer='adam', loss='binary_crossentropy')

# # Train the model
# siamese_network.train(x_train, y_train, x_val, y_val, batch_size=32, epochs=10)

# # Plot training history
# siamese_network.plot_training_history()

# # Optionally, evaluate the model on a test set prepared in a similar way
# # siamese_network.evaluate_model(x_test, y_test)
